This is my test file

In [2]:

import numpy as np
import pandas as pd
import geopandas as gpd
import plotly.figure_factory as ff


from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import silhouette_score
from sklearn.datasets import load_iris, load_wine
from sklearn.cluster import DBSCAN

random_state = 11

from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LogColorMapper,
    Range1d, CustomJS, Slider
)
from bokeh.palettes import Inferno256 as palette
from bokeh.plotting import figure
from bokeh.models import ColorMapper

from bokeh.layouts import row, widgetbox

In [3]:
df = pd.read_csv('census.csv')

In [4]:
df[df['county'].str.contains('Richmond')]

,Unnamed: 0,id,county,state,Estimate!!SEX AND AGE!!Total population!!65 years and over!!Sex ratio (males per 100 females),Estimate!!RACE!!Total population,Estimate!!RACE!!Total population!!One race,Estimate!!RACE!!Total population!!Two or more races,Estimate!!RACE!!Total population!!One race.1,Estimate!!RACE!!Total population!!One race!!White,...,Estimate!!SEX AND AGE!!Total population!!21 years and over,Estimate!!SEX AND AGE!!Total population!!62 years and over,Estimate!!SEX AND AGE!!Total population!!65 years and over,Estimate!!SEX AND AGE!!Total population!!18 years and over.1,Estimate!!SEX AND AGE!!Total population!!18 years and over!!Male,Estimate!!SEX AND AGE!!Total population!!18 years and over!!Female,Estimate!!SEX AND AGE!!Total population!!18 years and over!!Sex ratio (males per 100 females),Estimate!!SEX AND AGE!!Total population!!65 years and over.1,Estimate!!SEX AND AGE!!Total population!!65 years and over!!Male,Estimate!!SEX AND AGE!!Total population!!65 years and over!!Female
507,507,0500000US13245,Richmond County,Georgia,70.9,201463,196804,4659,196804,76073,...,144985,34309,26601,154550,73613,80937,91.0,26601,11039,15562
1870,1870,0500000US36085,Richmond County,New York,74.8,474101,462311,11790,462311,352259,...,353274,92049,73325,369598,175847,193751,90.8,73325,31369,41956
1966,1966,0500000US37153,Richmond County,North Carolina,77.3,45189,44439,750,44439,28209,...,32771,9562,7692,34731,16751,17980,93.2,7692,3353,4339
2895,2895,0500000US51159,Richmond County,Virginia,64.8,8878,8461,417,8461,5745,...,7193,2213,1846,7381,4241,3140,135.1,1846,726,1120
2944,2944,0500000US51760,Richmond city,Virginia,70.3,223787,216385,7402,216385,101498,...,170520,35516,27831,183783,85989,97794,87.9,27831,11489,16342


In [5]:
df['state'].replace({
    ' Alabama': 'AL',
    ' Alaska': 'AK',
    ' American Samoa': 'AS',
    ' Arizona': 'AZ',
    ' Arkansas': 'AR',
    ' California': 'CA',
    ' Colorado': 'CO',
    ' Connecticut': 'CT',
    ' Delaware': 'DE',
    ' District of Columbia': 'DC',
    ' Florida': 'FL',
    ' Georgia': 'GA',
    ' Guam': 'GU',
    ' Hawaii': 'HI',
    ' Idaho': 'ID',
    ' Illinois': 'IL',
    ' Indiana': 'IN',
    ' Iowa': 'IA',
    ' Kansas': 'KS',
    ' Kentucky': 'KY',
    ' Louisiana': 'LA',
    ' Maine': 'ME',
    ' Maryland': 'MD',
    ' Massachusetts': 'MA',
    ' Michigan': 'MI',
    ' Minnesota': 'MN',
    ' Mississippi': 'MS',
    ' Missouri': 'MO',
    ' Montana': 'MT',
    ' Nebraska': 'NE',
    ' Nevada': 'NV',
    ' New Hampshire': 'NH',
    ' New Jersey': 'NJ',
    ' New Mexico': 'NM',
    ' New York': 'NY',
    ' North Carolina': 'NC',
    ' North Dakota': 'ND',
    ' Northern Mariana Islands':'MP',
    ' Ohio': 'OH',
    ' Oklahoma': 'OK',
    ' Oregon': 'OR',
    ' Pennsylvania': 'PA',
    ' Puerto Rico': 'PR',
    ' Rhode Island': 'RI',
    ' South Carolina': 'SC',
    ' South Dakota': 'SD',
    ' Tennessee': 'TN',
    ' Texas': 'TX',
    ' Utah': 'UT',
    ' Vermont': 'VT',
    ' Virginia': 'VA',
    ' Washington': 'WA',
    ' West Virginia': 'WV',
    ' Wisconsin': 'WI',
    ' Wyoming': 'WY'} , inplace = True) 

In [6]:
df.rename( columns = {'id': "fips"}, inplace = True)

In [7]:
df.head()

,Unnamed: 0,fips,county,state,Estimate!!SEX AND AGE!!Total population!!65 years and over!!Sex ratio (males per 100 females),Estimate!!RACE!!Total population,Estimate!!RACE!!Total population!!One race,Estimate!!RACE!!Total population!!Two or more races,Estimate!!RACE!!Total population!!One race.1,Estimate!!RACE!!Total population!!One race!!White,...,Estimate!!SEX AND AGE!!Total population!!21 years and over,Estimate!!SEX AND AGE!!Total population!!62 years and over,Estimate!!SEX AND AGE!!Total population!!65 years and over,Estimate!!SEX AND AGE!!Total population!!18 years and over.1,Estimate!!SEX AND AGE!!Total population!!18 years and over!!Male,Estimate!!SEX AND AGE!!Total population!!18 years and over!!Female,Estimate!!SEX AND AGE!!Total population!!18 years and over!!Sex ratio (males per 100 females),Estimate!!SEX AND AGE!!Total population!!65 years and over.1,Estimate!!SEX AND AGE!!Total population!!65 years and over!!Male,Estimate!!SEX AND AGE!!Total population!!65 years and over!!Female
0,0,0500000US01001,Autauga County,AL,76.5,55200,54170,1030,54170,42437,...,39614,9403,8050,41831,20054,21777,92.1,8050,3488,4562
1,1,0500000US01003,Baldwin County,AL,86.6,208107,204535,3572,204535,179526,...,155502,49811,40665,162430,78104,84326,92.6,40665,18870,21795
2,2,0500000US01005,Barbour County,AL,76.2,25782,25429,353,25429,12216,...,19394,5658,4634,20346,10923,9423,115.9,4634,2004,2630
3,3,0500000US01007,Bibb County,AL,80.5,22527,22340,187,22340,17268,...,16864,4323,3661,17868,9578,8290,115.5,3661,1633,2028
4,4,0500000US01009,Blount County,AL,79.7,57645,56710,935,56710,55054,...,42232,12400,10233,44177,21595,22582,95.6,10233,4540,5693


In [8]:
fip = df['fips'].str.split('US',expand = True)[1]

In [9]:
fip

0       01001
1       01003
2       01005
3       01007
4       01009
        ...  
3215    72145
3216    72147
3217    72149
3218    72151
3219    72153
Name: 1, Length: 3220, dtype: object

In [10]:
df['fip'] = fip

In [11]:
df.drop(columns = ['Unnamed: 0'], inplace = True)

In [12]:
cols = list(df)

In [13]:
cols.insert(0,cols.pop(cols.index('fip')))

In [14]:
df = df.loc[:,cols]

In [15]:
df.drop(columns = 'fips', inplace = True)

In [16]:
df.drop(df.loc[df['state'] == 'AK'].index, inplace=True)
df.drop(df.loc[df['state'] == 'HI'].index, inplace=True)
df.drop(df.loc[df['state'] == 'PR'].index, inplace=True)


In [17]:
df.shape

(3108, 92)

In [18]:
contig = gpd.read_file('cb_2018_us_county_500k.shp')
values = range(len(fip))

In [19]:
contiguous_usa = gpd.read_file('cb_2018_us_county_500k.shp')
contiguous_usa[contiguous_usa['NAME'] == 'New York']
geo = contiguous_usa

In [20]:
geo.shape

(3233, 10)

In [21]:
geo = geo[(geo['STATEFP'] != '15') & (geo['STATEFP'] != '02') & (geo['STATEFP'] != '72') & 
           (geo['STATEFP'] != '66') & (geo['STATEFP'] != '78') & (geo['STATEFP'] != '69') &
           (geo['STATEFP'] != '60')]

In [22]:
master = pd.read_csv('../../../area_included.csv')

In [24]:
X = master.drop(columns = ['total_covid', 'deaths', 'county_x', 'county_y', 'state', 'id', 'population', 
                           'beds_per_case', 'unnamed: 0_y', 'st_num',
                           'cases', 'countyfips', 'county'])


In [25]:
ss = StandardScaler()

X_scaled = ss.fit_transform(X)

dbscan = DBSCAN(eps = 1, min_samples= 5)
dbscan.fit(X_scaled)

DBSCAN(algorithm='auto', eps=1, leaf_size=30, metric='euclidean',
       metric_params=None, min_samples=5, n_jobs=None, p=None)

In [26]:
geo['cluster'] = dbscan.labels_

In [27]:
geo['cluster'].value_counts()

-1     1369
 0     1351
 1      293
 2       31
 5       20
 8        7
 10       6
 6        6
 11       5
 9        5
 7        5
 3        5
 4        5
Name: cluster, dtype: int64

In [32]:
import json
from bokeh.io import show
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          CustomJS, CustomJSFilter, 
                          GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.plotting import figure

geosource = GeoJSONDataSource(geojson = geo.to_json())

In [34]:
color_mapper = LinearColorMapper(palette = palette, low = -1, high = 13)
# Create figure object.
p = figure(title = 'Testing Map', 
           plot_height = 600 ,
           plot_width = 950, 
           toolbar_location = 'below',
           tools = "pan, wheel_zoom, box_zoom, reset")
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None


counties = p.patches('xs', 'ys', source = geosource,
                    fill_color = {'field' : 'cluster',
                                 'transform' : color_mapper},
                    line_color = 'gray',
                    line_width = 0.25,
                    fill_alpha = 1)


p.add_tools(HoverTool(renderers = [counties],
                     tooltips =[('county', '@NAME'),
                               ('state', '@STATEFP'),
                               ('Cluster', '@cluster')]))
# Define color palettes
palette = brewer['Pastel1'][8]
palette = palette[::-1] # reverse order of colors so higher values have darker colors                     
    
color_mapper = LinearColorMapper(palette = palette, low = -1, high = 13)
    
color_bar = ColorBar(color_mapper = color_mapper,
                    label_standoff = 8,
                    width = 500,
                    height = 20,
                    border_line_color = None,
                    location = (0,0),
                    orientation = 'horizontal')    



show(p)